In [ ]:
import sys
sys.path.insert(0, "/Users/jisoo/projects/thesis/carte_test")

from config import RAW, PROCESSED

In [ ]:
from __future__ import annotations

from pathlib import Path
import duckdb


def filter_ratings_recent200_u20(
    ratings_csv: Path,
    out_parquet: Path,
    max_recent_per_user: int = 200,
    min_items_per_user: int = 20,
) -> None:
    """
    - 목적
      - ratings.csv에서
        1) user별 최근 max_recent_per_user개만 유지
        2) 그 결과 길이가 min_items_per_user 이상인 user만 유지
      - 결과를 parquet로 저장

    - 출력 parquet 컬럼
      - userId, movieId, rating, timestamp
    """
    out_parquet.parent.mkdir(parents=True, exist_ok=True)

    con = duckdb.connect(database=":memory:")

    # - CSV 로드
    con.execute(
        f"""
        CREATE OR REPLACE TABLE ratings_raw AS
        SELECT
            CAST(userId AS INTEGER)     AS userId,
            CAST(movieId AS INTEGER)    AS movieId,
            CAST(rating AS FLOAT)       AS rating,
            CAST(timestamp AS BIGINT)   AS timestamp
        FROM read_csv_auto('{ratings_csv.as_posix()}',
                           header=True,
                           sample_size=100000);
        """
    )

    # - 최근 200개 유지 → 길이 20 이상만 유지
    # - timestamp 동률이 있을 수 있으니 tie-breaker로 movieId를 같이 사용
    con.execute(
        f"""
        CREATE OR REPLACE TABLE ratings_filtered AS
        WITH ranked AS (
            SELECT
                *,
                row_number() OVER (
                    PARTITION BY userId
                    ORDER BY timestamp DESC, movieId DESC
                ) AS rn_desc
            FROM ratings_raw
        ),
        recent AS (
            SELECT
                userId, movieId, rating, timestamp
            FROM ranked
            WHERE rn_desc <= {max_recent_per_user}
        ),
        with_len AS (
            SELECT
                *,
                count(*) OVER (PARTITION BY userId) AS seq_len
            FROM recent
        )
        SELECT
            userId, movieId, rating, timestamp
        FROM with_len
        WHERE seq_len >= {min_items_per_user};
        """
    )

    # - parquet 저장
    con.execute(f"COPY ratings_filtered TO '{out_parquet.as_posix()}' (FORMAT PARQUET);")

    con.close()


def print_simple_stats_from_parquet(
    ratings_parquet: Path,
) -> None:
    """
    - 목적
      - 필터링된 parquet에서 남은 유저 수/row 수 + 유저당 길이 요약 통계를 출력
    - DuckDB로 바로 집계해서 pandas 로딩 없이도 빠름
    """
    con = duckdb.connect(database=":memory:")

    # - 유저 수, row 수
    base = con.execute(
        f"""
        SELECT
            COUNT(*) AS num_rows,
            COUNT(DISTINCT userId) AS num_users
        FROM read_parquet('{ratings_parquet.as_posix()}');
        """
    ).fetchone()

    num_rows = int(base[0])
    num_users = int(base[1])

    print("========== [Filtered Ratings Stats] ==========")
    print(f"- num_users : {num_users:,}")
    print(f"- num_rows  : {num_rows:,}")

    # - 유저별 길이 분포 요약
    #   - DuckDB quantile_cont로 percentiles 계산
    dist = con.execute(
        f"""
        WITH user_len AS (
            SELECT userId, COUNT(*) AS seq_len
            FROM read_parquet('{ratings_parquet.as_posix()}')
            GROUP BY userId
        )
        SELECT
            MIN(seq_len) AS min_len,
            AVG(seq_len) AS mean_len,
            quantile_cont(seq_len, 0.50) AS p50_len,
            quantile_cont(seq_len, 0.90) AS p90_len,
            quantile_cont(seq_len, 0.95) AS p95_len,
            quantile_cont(seq_len, 0.99) AS p99_len,
            MAX(seq_len) AS max_len
        FROM user_len;
        """
    ).fetchone()

    print("\n[Per-user sequence length]")
    print(f"- min  : {int(dist[0]):,}")
    print(f"- mean : {float(dist[1]):.2f}")
    print(f"- p50  : {int(dist[2]):,}")
    print(f"- p90  : {int(dist[3]):,}")
    print(f"- p95  : {int(dist[4]):,}")
    print(f"- p99  : {int(dist[5]):,}")
    print(f"- max  : {int(dist[6]):,}")

    con.close()

import random
from typing import Optional


def pick_random_user_id(
    con: duckdb.DuckDBPyConnection,
    ratings_parquet: Path,
    seed: int = 42,
) -> int:
    """
    - 목적
      - parquet에서 임의의 userId 1개 뽑기
    - 구현 포인트
      - COUNT(DISTINCT) 후 random offset으로 1개 선택
      - 완전 균일 샘플링은 아니지만(내부 구현상), 디버깅/샘플 출력 용도로 충분
    """
    random.seed(seed)

    # - user 수
    num_users = con.execute(
        f"""
        SELECT COUNT(DISTINCT userId)
        FROM read_parquet('{ratings_parquet.as_posix()}');
        """
    ).fetchone()[0]
    num_users = int(num_users)

    # - 랜덤 오프셋
    offset = random.randint(0, max(0, num_users - 1))

    # - DISTINCT userId 목록에서 offset으로 1개 선택
    user_id = con.execute(
        f"""
        SELECT userId
        FROM (
            SELECT DISTINCT userId
            FROM read_parquet('{ratings_parquet.as_posix()}')
            ORDER BY userId
        )
        LIMIT 1 OFFSET {offset};
        """
    ).fetchone()[0]

    return int(user_id)


def print_user_sequence_sample(
    ratings_parquet: Path,
    user_id: Optional[int] = None,
    limit: int = 30,
    seed: int = 42,
) -> None:
    """
    - 목적
      - (1) user_id가 주어지면 해당 유저의 시퀀스 출력
      - (2) user_id가 None이면 임의 유저 1명을 뽑아서 출력
    - 출력
      - pos, timestamp, rating, movieId (timestamp 오름차순)
    """
    con = duckdb.connect(database=":memory:")

    # - user_id 미지정이면 랜덤으로 선택
    if user_id is None:
        user_id = pick_random_user_id(con=con, ratings_parquet=ratings_parquet, seed=seed)

    # - 유저 시퀀스 길이
    seq_len = con.execute(
        f"""
        SELECT COUNT(*) AS seq_len
        FROM read_parquet('{ratings_parquet.as_posix()}')
        WHERE userId = {user_id};
        """
    ).fetchone()[0]
    seq_len = int(seq_len)

    print("\n========== [User Sequence Sample] ==========")
    print(f"- userId : {user_id}")
    print(f"- seq_len: {seq_len}")
    print(f"- showing first {min(limit, seq_len)} items (time ascending)\n")

    # - timestamp 오름차순으로 pos 부여해서 출력
    rows = con.execute(
        f"""
        SELECT
            row_number() OVER (ORDER BY timestamp ASC, movieId ASC) - 1 AS pos,
            timestamp,
            strftime(to_timestamp(timestamp), '%Y-%m-%d %H:%M:%S') AS datetime_utc,
            rating,
            movieId
        FROM read_parquet('{ratings_parquet.as_posix()}')
        WHERE userId = {user_id}
        ORDER BY timestamp ASC, movieId ASC
        LIMIT {limit};
        """
    ).fetchall()

    # - 보기 좋게 출력
    print("pos\ttimestamp\tdatetime_utc\t\trating\tmovieId")
    for pos, ts, dt, r, mid in rows:
        print(f"{int(pos)}\t{int(ts)}\t{dt}\t{float(r):.1f}\t{int(mid)}")

    # - 마지막 몇 개도 보고 싶을 때 참고용 안내
    if seq_len > limit:
        print(f"\n... (truncated) total={seq_len}, increase limit to see more.")

    con.close()


# =============================================================================
# 실행
# =============================================================================
# 1) 필터링 저장
filter_ratings_recent200_u20(
    ratings_csv=RAW.RATINGS_CSV,
    out_parquet=PROCESSED.RATINGS_PARQUET,
    max_recent_per_user=100,
    min_items_per_user=20,
)

# 2) 간단 통계 출력
print_simple_stats_from_parquet(PROCESSED.RATINGS_PARQUET)

# 3) 임의 유저 시퀀스 출력 (랜덤 1명)
print_user_sequence_sample(
    ratings_parquet=PROCESSED.RATINGS_PARQUET,
    user_id=None,   # 특정 유저 보고 싶으면 숫자로 넣기
    limit=30,
    seed=42,
)